# Stack loading
---

### Overview
Explore image reading with `dask-image` and optimize procedure for loading image stacks

In [1]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import dask as da

from skimage import io, data, exposure

from dask_image.imread import imread
import dask_image.ndfilters as ndfilters
import dask_image.ndmeasure as ndmeasure

### Load a stack with `dask-image`

In [3]:
# Set data directory
data_dir = 'D:/rlane/Data/zstacks/2021-08-05-12-36-33-zstack-28.56deg_3.8776rad/'

# Load data
stack = imread(data_dir + '/*.tif')
stack

dask.array<_map_read_frame, shape=(161, 2048, 2048), dtype=uint16, chunksize=(1, 2048, 2048), chunktype=numpy.ndarray>

### Simple computation
As image stack is not loaded into RAM, simple computations (if not threaded) take forevah

In [4]:
stack.mean()  # just considers doing the computation

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [5]:
stack.mean().compute()  # actually does the computation

108.53365936338531

In [6]:
# Takes forever if not parallelized
%timeit -r 3 -n 1 stack.mean().compute()

2.76 s ± 30.4 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### Rescale
How would do you something like rescaling the stack to a 32bit float

In [7]:
stack_f = stack.astype(np.float32)
stack_f

dask.array<astype, shape=(161, 2048, 2048), dtype=float32, chunksize=(1, 2048, 2048), chunktype=numpy.ndarray>

In [8]:
stack_n = (stack_f - stack_f.min()) / (stack_f - stack_f.min()).max()
stack_n

dask.array<truediv, shape=(161, 2048, 2048), dtype=float32, chunksize=(1, 2048, 2048), chunktype=numpy.ndarray>

In [9]:
# Check data range
stack_n.min().compute(), stack_n.max().compute()

(0.0, 1.0)

In [10]:
# Still slow
%timeit -r 1 -n 1 stack_n.min().compute(), stack_n.max().compute()

8.61 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Rechunking

Can easily "rechunk" the stack

In [14]:
stack.rechunk(chunks=(30, 2048, 2048, -1))

dask.array<rechunk-merge, shape=(161, 2048, 2048), dtype=uint16, chunksize=(30, 2048, 2048), chunktype=numpy.ndarray>

or even store the whole stack in a single chunk

In [16]:
stack.rechunk(chunks=(-1, 2048, 2048, 1))

dask.array<rechunk-merge, shape=(161, 2048, 2048), dtype=uint16, chunksize=(161, 2048, 2048), chunktype=numpy.ndarray>

### Parallelizizing


In [20]:
from dask.distributed import Client

client = Client(n_workers=4)

C:\Users\rlane\AppData\Local\Continuum\miniconda3\envs\psf\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56332 instead
  warnings.warn(
